<style>
    div.container {
      max-width: 800px!important;
    }
</style>

# SparkML Basics

In [1]:
def init: Unit = {
  import ammonite.ops._
  val jars = ls! root/'opt/'spark/'jars |? (_.ext == "jar")
  jars.foreach(interp.load.cp(_))   
}

init

defined function init

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.ERROR)

import org.apache.log4j.{Level, Logger}


In [3]:
import org.apache.spark.sql._

import org.apache.spark.sql._

In [4]:
val spark = SparkSession
  .builder
  .config("hive.metastore.uris","thrift://localhost:9083") 
  .config("spark.sql.warehouse.dir", "/data/hive/warehouse")
  .master("local[*]")
  .appName("Spark SQL Basics")
  .enableHiveSupport()
  .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


spark: SparkSession = org.apache.spark.sql.SparkSession@6e04003e

In [5]:
val sc = spark.sparkContext

sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@2e0bcb77

In [43]:
val iris = spark
  .read
  .format("csv")
  .option("header", "true")
  .option("delimiter", ",")
  .option("inferSchema", "true")
  .load("file:///data/csv/iris.csv")
  .withColumnRenamed("Sepal.Length", "sepalLength")
  .withColumnRenamed("Sepal.Width", "sepalWidth")
  .withColumnRenamed("Petal.Length", "petalLength")
  .withColumnRenamed("Petal.Width", "petalWidth")
  
iris.limit(5).show

+-----------+----------+-----------+----------+-------+
|sepalLength|sepalWidth|petalLength|petalWidth|Species|
+-----------+----------+-----------+----------+-------+
|        5.1|       3.5|        1.4|       0.2| setosa|
|        4.9|       3.0|        1.4|       0.2| setosa|
|        4.7|       3.2|        1.3|       0.2| setosa|
|        4.6|       3.1|        1.5|       0.2| setosa|
|        5.0|       3.6|        1.4|       0.2| setosa|
+-----------+----------+-----------+----------+-------+



iris: DataFrame = [sepalLength: double, sepalWidth: double ... 3 more fields]

In [44]:
val List(train, test) = iris.randomSplit(Array(0.8, 0.2), 1).toList

train: Dataset[Row] = [sepalLength: double, sepalWidth: double ... 3 more fields]
test: Dataset[Row] = [sepalLength: double, sepalWidth: double ... 3 more fields]

In [12]:
train.count

res11: Long = 118L

In [13]:
test.count

res12: Long = 32L

In [39]:
val labelIndexer = new StringIndexer()
  .setInputCol("Species")
  .setOutputCol("label")

val d1 = labelIndexer.fit(iris).transform(iris)

d1.limit(5).show

+-----------+----------+-----------+----------+-------+-----+
|sepalLength|sepalWidth|petalLength|petalWidth|Species|label|
+-----------+----------+-----------+----------+-------+-----+
|        5.1|       3.5|        1.4|       0.2| setosa|  2.0|
|        4.9|       3.0|        1.4|       0.2| setosa|  2.0|
|        4.7|       3.2|        1.3|       0.2| setosa|  2.0|
|        4.6|       3.1|        1.5|       0.2| setosa|  2.0|
|        5.0|       3.6|        1.4|       0.2| setosa|  2.0|
+-----------+----------+-----------+----------+-------+-----+



labelIndexer: StringIndexer = strIdx_b26f47000635
d1: DataFrame = [sepalLength: double, sepalWidth: double ... 4 more fields]

In [41]:
val featureIndexer = new VectorAssembler()
  .setInputCols(Array("sepalLength", "sepalWidth", "petalLength", "petalWidth"))
  .setOutputCol("features")
  
val d2 = featureIndexer.transform(d1)

d2.limit(5).show

+-----------+----------+-----------+----------+-------+-----+-----------------+
|sepalLength|sepalWidth|petalLength|petalWidth|Species|label|         features|
+-----------+----------+-----------+----------+-------+-----+-----------------+
|        5.1|       3.5|        1.4|       0.2| setosa|  2.0|[5.1,3.5,1.4,0.2]|
|        4.9|       3.0|        1.4|       0.2| setosa|  2.0|[4.9,3.0,1.4,0.2]|
|        4.7|       3.2|        1.3|       0.2| setosa|  2.0|[4.7,3.2,1.3,0.2]|
|        4.6|       3.1|        1.5|       0.2| setosa|  2.0|[4.6,3.1,1.5,0.2]|
|        5.0|       3.6|        1.4|       0.2| setosa|  2.0|[5.0,3.6,1.4,0.2]|
+-----------+----------+-----------+----------+-------+-----+-----------------+



featureIndexer: VectorAssembler = vecAssembler_d4a6aa201603
d2: DataFrame = [sepalLength: double, sepalWidth: double ... 5 more fields]

In [42]:
import org.apache.spark.ml.feature.{StringIndexer, VectorAssembler, IndexToString}
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.evaluation.ClusteringEvaluator

val kmeans = new KMeans().setK(3).setSeed(1L)
val model = kmeans.fit(d2)

val predictions = model.transform(d2)

val evaluator = new ClusteringEvaluator()
val silhouette = evaluator.evaluate(predictions)
println(s"Silhouette with squared euclidean distance = $silhouette")

println("Cluster Centers: ")
model.clusterCenters.foreach(println)

19/07/25 21:14:01 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
19/07/25 21:14:01 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


Silhouette with squared euclidean distance = 0.7344130579787816
Cluster Centers: 
[5.88360655737705,2.7409836065573776,4.388524590163936,1.4344262295081969]
[6.853846153846153,3.0769230769230766,5.715384615384615,2.053846153846153]
[5.005999999999999,3.428000000000001,1.4620000000000002,0.2459999999999999]


kmeans: KMeans = kmeans_44e7f4b624f5
model: org.apache.spark.ml.clustering.KMeansModel = kmeans_44e7f4b624f5
predictions: DataFrame = [sepalLength: double, sepalWidth: double ... 6 more fields]
evaluator: ClusteringEvaluator = cluEval_d58edc4951d5
silhouette: Double = 0.7344130579787816